## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embeddings, load one from gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2025-01-18 01:07:31--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6026:18::a27d:4612, 162.125.71.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6026:18::a27d:4612|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2025-01-18 01:07:31--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc05f3a6ec6fa08ece05878ac922.dl.dropboxusercontent.com/cd/0/inline/CiZx1p6PI3Q1Jsqa5u2AGTmlHd4fQFLwPV9IKfooG-Ihc3t34KatglrTD8TiIO5nGBTuMxMVN2PmCDyn_LUrnu07i0b4NrZZZ-LQzlAXSyjTJPY6ta1zTVCtjXDtf3Pl7g4/file?dl=1# [following]
--2025-01-18 01:07:32--  https://uc05f3a6ec6fa08ece05878ac922.dl.dropboxusercontent.com/cd/0

In [1]:
import numpy as np

with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[50: 52]

["What TV shows or books help you read people's body language?\n",
 "I see this cute girl around my college. I'm too shy to approach her. Is it acceptable to leave her a note with my feelings and number?\n"]

__Tokenization:__ a typical first step for an NLP task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many NLP tasks like tokenization, stemming or part-of-speech tagging.

In [2]:
pip install --upgrade nltk gensim bokeh

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [4]:
data[1:5]

['What are some ways to overcome a fast food addiction?\n',
 'Who were the great Chinese soldiers and leaders who fought in WW2?\n',
 'What are ZIP codes in the Bay Area?\n',
 'Why was George RR Martin critical of JK Rowling after losing the Hugo award?\n']

In [5]:
len(data)

537272

In [6]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(sent.lower()) for sent in data]

In [7]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [8]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [6]:
pip install gensim

  Using cached numpy-1.26.4-cp311-cp311-macosx_10_9_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 7.0 MB/s eta 0:00:0000:0100:01
Using cached numpy-1.26.4-cp311-cp311-macosx_10_9_x86_64.whl (20.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 5.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.1
    Uninstalling numpy-2.2.1:
      Successfully uninstalled numpy-2.2.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.1
    Uninstalling scipy-1.15.1:
      Successfully uninstalled scipy-1.15.1
Note: you may need to restart the kernel to use updated packages.


In [9]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

# From gensim docs
# vw: This object essentially contains the mapping between words and embeddings.
# After training, it can be used directly to query those embeddings in various ways.

In [10]:
# now you can get word vectors !
model.get_vector('anything')

array([-3.1864421 ,  1.9026654 ,  0.31908998,  2.557702  ,  2.0852659 ,
        2.1173706 ,  1.6642973 , -4.7403865 ,  0.31088987,  2.2769172 ,
       -0.4662609 ,  1.8990656 ,  3.1537845 , -0.17679642,  2.7787945 ,
       -1.9080304 ,  0.04209351, -1.3670801 ,  0.37552458, -1.6339743 ,
       -2.1002302 , -0.38412663, -1.3313946 , -3.8927157 ,  2.5245645 ,
       -2.4400723 ,  0.35729972,  1.7179115 ,  0.75446284,  0.41571006,
        0.6648478 ,  0.723424  ], dtype=float32)

In [12]:
model.most_similar('bread')

[('rice', 0.9595727324485779),
 ('cheese', 0.9412733316421509),
 ('butter', 0.9377203583717346),
 ('sauce', 0.9339877367019653),
 ('beans', 0.9218798279762268),
 ('wine', 0.9147394299507141),
 ('vodka', 0.9138975143432617),
 ('corn', 0.9138529300689697),
 ('banana', 0.9134515523910522),
 ('chocolate', 0.9107043743133545)]

In [13]:
# or query similar words directly. Go play with it!
model.most_similar('vodka')

[('rice', 0.9194060564041138),
 ('wine', 0.9140605926513672),
 ('bread', 0.9138974547386169),
 ('chocolate', 0.9017302393913269),
 ('potato', 0.8998499512672424),
 ('honey', 0.8944750428199768),
 ('beans', 0.8937272429466248),
 ('rum', 0.8929306268692017),
 ('tequila', 0.8927705883979797),
 ('cheese', 0.8910321593284607)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

After being downloaded for the first time (or if you manually delete it), the model is saved in the `~/gensim_data` or `%USER_PATH%/gensim_data` directory. This can be checked seting the return_path parameter to True.

In [ ]:
git clone gensim and rename list json
ln -s /etc/ssl/* /Library/Frameworks/Python.framework/Versions/3.11/etc/openssl/

In [14]:
import gensim.downloader as api
# api.load('glove-wiki-gigaword-50')

# api.load('glove-wiki-gigaword-100')
model = api.load('glove-twitter-100')

In [15]:
model.most_similar(positive=["coder", "money"], negative=["brain"])
# model.most_similar(positive=["певец", "купец"], negative=["следопыт"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.4922019839286804),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [18]:
words = model.index_to_key[:10000] 

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei', 'mesmo', 'ainda', 'bang', 'radio', 'sounds', 'kinda', 'sekolah', 'dicen', 'hehehe', 'lead', 'الف', 'basketball', 'hotel', 'despues', 'din', 'earth', 'email', 'sh', 'jan', 'mucha', 'fais', 'airport', 'stfu', 'rato', 'ناس', 'شر', 'lls', 'area', 'dai', 'tenia', 'fox', 'emotional', 'scene', 'create', 'makanya', 'common', 'colombia', 'budak', 'جديد', 'και', 'gata', 'twerk', 'ketawa', 'strength', 'صحيح', 'sd', 'pilih', 'extremely', 'becomes', 'agak', 'kayo', 'rica', 'anger', 'bosen', 'hunter', 'ninja', 'frozen', 'maldita', 'adult', 'tiket', 'allá', 'tuk', 'eggs', 'ping', 'naon', 'monsters', 'drank', 'hape', 'aver', 'charles', 'rage', 'besides', 'lifetime', 'excuses', 'النفس', 'yaah', 'عنها', 'nine', 'pang', 'advantage', 'nouveau', 'teens', 'cium', 'bone', 'tracks', 'loro', 'miercoles', 'hunting', 'secretly', 'cumplir']


In [19]:
len(words)

10000

In [20]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(word) for word in words])

In [21]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is **P**rincipial **C**omponent **A**nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [51]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


fit(X, y=None)[source]
Fit the model with X.

Parameters
:
X
{array-like, sparse matrix} of shape (n_samples, n_features)
Training data, where n_samples is the number of samples and n_features is the number of features.

y
Ignored
Ignored.

Returns
:
self
object
Returns the instance itself.

fit_transform(X, y=None)[source]
Fit the model with X and apply the dimensionality reduction on X.

Parameters
:
X
{array-like, sparse matrix} of shape (n_samples, n_features)
Training data, where n_samples is the number of samples and n_features is the number of features.

y
Ignored
Ignored.

Returns
:
X_new
ndarray of shape (n_samples, n_components)
Transformed values.

In [22]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = pca.fit_transform(word_vectors)

# and maybe MORE OF YOUR CODE here :)

In [24]:
word_vectors

array([[ 0.63006 ,  0.65177 ,  0.25545 , ...,  0.55096 ,  0.64706 ,
        -0.6093  ],
       [ 0.18205 , -0.048483,  0.23966 , ..., -0.3358  ,  0.18884 ,
        -0.40786 ],
       [ 1.0674  ,  0.45716 ,  0.51463 , ...,  0.13974 ,  0.76487 ,
        -0.17314 ],
       ...,
       [-0.034233,  1.7685  ,  0.71839 , ..., -1.6638  , -0.57892 ,
         0.13035 ],
       [ 0.60622 , -0.84738 ,  0.27267 , ...,  0.29761 , -0.55103 ,
         0.38739 ],
       [ 0.14805 ,  0.89738 ,  0.78109 , ...,  0.79974 ,  0.76872 ,
         1.2261  ]], dtype=float32)

In [23]:
word_vectors_pca

array([[ 0.01445404,  0.25714645],
       [-0.01883862,  0.08148453],
       [-0.35034105,  0.29866126],
       ...,
       [-1.0320995 ,  3.2098746 ],
       [-1.8794451 , -1.6018338 ],
       [-0.12088823,  0.3235639 ]], dtype=float32)

In [25]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
# assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

In [55]:
pip install bokeh

Note: you may need to restart the kernel to use updated packages.


#### Let's draw it!

In [26]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [27]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [61]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca

tsne = TSNE(n_components=2, perplexity=3)
word_tsne = tsne.fit_transform(word_vectors)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [62]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1106', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [46]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    phrase_l  = phrase.lower()
    # print(phrase_l)
    tokens = tokenizer.tokenize(phrase_l)
    # print(tokens)
    
    words_vectors = np.array([model.get_vector(word) for word in words if word in model.index_to_key])
    # print(word_vectors)
    
    
    vector = np.mean(word_vectors, axis=0)
    # print(vector, vector.shape )
    if not vector.all():
        vector = np.zeros([model.vector_size], dtype='float32')
    
    
    return vector
        
    

In [50]:
get_phrase_embedding("thisisgibberish")

array([ 9.08862725e-02,  6.87497854e-02,  7.02829612e-03,  4.80612129e-04,
       -4.09741811e-02,  3.78008895e-02,  6.41432256e-02,  7.14342594e-02,
        1.59367919e-02,  1.20861903e-01, -2.57156827e-02, -1.78476945e-02,
       -1.48830462e+00,  1.20494356e-02, -1.16013773e-02,  4.70957048e-02,
       -1.11720115e-02, -6.24956861e-02, -1.22895285e-01, -3.93887311e-02,
       -4.51822579e-03,  3.52527620e-03,  1.21910416e-04,  2.40479540e-02,
        4.76355441e-02, -8.23999882e-01, -2.20572613e-02, -3.49956453e-02,
        6.55393451e-02,  1.81521364e-02, -6.02833442e-02, -3.76678146e-02,
       -8.01329091e-02,  1.44663602e-02,  1.86161354e-01,  4.49868143e-02,
        2.14139652e-02, -2.70996131e-02, -2.81668175e-02, -6.44954620e-03,
       -5.72430789e-01,  2.03284603e-02, -1.95757300e-02,  4.02626954e-02,
        5.85671924e-02, -4.58416045e-02, -3.84433568e-02,  1.25492373e-02,
       -4.37639020e-02, -4.80157472e-02, -1.61600024e-01, -4.94714174e-03,
       -2.73263385e-03, -

In [49]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

# assert np.allclose(vector[::10],
#                    np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
#                              -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
#                               dtype=np.float32))
assert np.array_equal(get_phrase_embedding("thisisgibberish"), np.zeros([model.vector_size], dtype='float32')), "corner case for all missing words should be handled as described in the function comments"

AssertionError: corner case for all missing words should be handled as described in the function comments

In [ ]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = # YOUR CODE

In [ ]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [ ]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [ ]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [ ]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [ ]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    
    return <YOUR CODE: top-k lines starting from most similar>

In [ ]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

In [ ]:
find_nearest(query="How does Trump?", k=10)

In [ ]:
find_nearest(query="Why don't i ask a question myself?", k=10)

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize `find_nearest` with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.